In [19]:
#################################################################################
# In this program, we shall be developing a movie Rec System
#################################################################################

'''
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

!pip install findspark

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
#spark = SparkSession.builder.master("local[4]")  # this creates spark session to  run locally, the braket[] means number of cores 
'''


In [1]:
# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [1]:

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Rec System") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
movies = spark.read.csv("movies.csv",header=True)
ratings = spark.read.csv("ratings.csv",header=True)

In [3]:
ratings.show()


+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [10]:

ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [4]:
#formatting our rating df

from pyspark.sql import functions as F

ratings = ratings.\
    withColumn('userId', F.col('userId').cast('integer')).\
    withColumn('movieId', F.col('movieId').cast('integer')).\
    withColumn('rating', F.col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



In [5]:
# Sparsity shows how sparse is the matrix, e.g. 99% sparsity shows that, 99% of the entries in the matrix are missing.
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.30% empty.


In [6]:
# Group data by userId, count ratings
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|   414| 2698|
|   599| 2478|
|   474| 2108|
|   448| 1864|
|   274| 1346|
|   610| 1302|
|    68| 1260|
|   380| 1218|
|   606| 1115|
|   288| 1055|
|   249| 1046|
|   387| 1027|
|   182|  977|
|   307|  975|
|   603|  943|
|   298|  939|
|   177|  904|
|   318|  879|
|   232|  862|
|   480|  836|
+------+-----+
only showing top 20 rows



In [8]:

# Group data by movieId, count ratings
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|   2858|  204|
|     50|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



In [ ]:
##################################
# ALS MODEL --- Alternative Least Square Method 

In [9]:

# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [10]:
# Split the ratings dataframe into training and test data
(train, test) = ratings.randomSplit([0.8, 0.2], seed=42)

# Set the ALS hyperparameters
als = ALS(
    userCol="userId", 
    itemCol="movieId", 
    ratingCol="rating", 
    rank =10, 
    maxIter =10, 
    regParam =.1,
    coldStartStrategy="drop", 
    nonnegative =True, 
    implicitPrefs = False
)

# Fit the model to the training_data
als_model = als.fit(train)

# Generate predictions on the test_data
test_predictions = als_model.transform(test)
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|   4896|   4.0| 3.4162953|
|   148|   5618|   3.0| 3.7065077|
|   148|   7153|   3.0| 3.4188726|
|   148|  40629|   5.0| 3.2049396|
|   148|  40815|   4.0| 3.5776799|
|   148|  60069|   4.5| 3.7646391|
|   148|  68954|   4.0|  3.833721|
|   148|  69844|   4.0|  3.527831|
|   148|  79132|   1.5| 3.8848894|
|   148|  79702|   4.0| 3.6236491|
|   148|  81834|   4.0| 3.5732112|
|   148|  81847|   4.5| 2.9250026|
|   148|  98243|   4.5| 3.8240626|
|   148|  98491|   5.0| 3.5541852|
|   148| 108932|   4.0| 3.6497884|
|   463|   1088|   3.5| 3.8528962|
|   463|   1221|   4.5| 3.9813879|
|   463|   2028|   4.5| 4.1558304|
|   463|   2167|   3.0| 3.8384085|
|   463|   3448|   3.0| 3.6221385|
+------+-------+------+----------+
only showing top 20 rows



In [12]:
# Import RegressionEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

# Complete the evaluator code
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# Extract the 3 parameters
print(evaluator.getMetricName())
print(evaluator.getLabelCol())
print(evaluator.getPredictionCol())


# Evaluate the "test_predictions" dataframe
RMSE = evaluator.evaluate(test_predictions)

# Print the RMSE
print ('RMSE = ', RMSE)

rmse
rating
prediction
RMSE =  0.8728464473016619


In [13]:
# Generate n Recommendations for all users
nrecommendations = als_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

C:\Users\moham\AppData\Local\Programs\Python\Python38\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{96004, 5.75677}...|
|     3|[{6835, 4.903592}...|
|     5|[{4642, 4.864058}...|
|     6|[{51931, 5.074078...|
|     9|[{96004, 5.376312...|
|    12|[{32892, 5.905121...|
|    13|[{53123, 5.331166...|
|    15|[{26171, 5.214606...|
|    16|[{25771, 4.497718...|
|    17|[{96004, 5.104533...|
+------+--------------------+



In [14]:
#Explaining the recommendations i.e splitting recommendations col to movie id and rating

from pyspark.sql.functions import split, explode

nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', F.col("rec_exp.movieId"), F.col("rec_exp.rating"))

nrecommendations.limit(11).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|     1|  96004|  5.75677|
|     1|   3379|  5.75677|
|     1| 171495|5.6745763|
|     1|  33649| 5.661552|
|     1| 177593| 5.646361|
|     1|  93988|5.5603123|
|     1|   7748| 5.515378|
|     1| 132333|5.5100846|
|     1|   5915|5.4998503|
|     1|  78836| 5.485668|
|     3|   6835| 4.903592|
+------+-------+---------+



In [15]:
#Explaining the recommendations 
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()


+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|  98279|   100|  5.14397|Fantastic Fear of...|              Comedy|
|   6732|   100| 5.110941|Hello, Dolly! (1969)|Comedy|Musical|Ro...|
|   3606|   100|5.0456553|  On the Town (1949)|Comedy|Musical|Ro...|
|  92643|   100|5.0208907|Monsieur Lazhar (...|Children|Comedy|D...|
|   1066|   100| 5.016412|Shall We Dance (1...|Comedy|Musical|Ro...|
|  74282|   100|  5.00795|Anne of Green Gab...|Children|Drama|Ro...|
|  67618|   100|5.0015407|Strictly Sexual (...|Comedy|Drama|Romance|
|  32892|   100|4.9997854|Ivan's Childhood ...|           Drama|War|
|    171|   100| 4.976853|      Jeffrey (1995)|        Comedy|Drama|
|     82|   100|4.9596453|Antonia's Line (A...|        Comedy|Drama|
+-------+------+---------+--------------------+--------------------+



In [23]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).limit(10).show()


+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   1101|   100|   5.0|      Top Gun (1986)|      Action|Romance|
|   1958|   100|   5.0|Terms of Endearme...|        Comedy|Drama|
|   2423|   100|   5.0|Christmas Vacatio...|              Comedy|
|   4041|   100|   5.0|Officer and a Gen...|       Drama|Romance|
|   5620|   100|   5.0|Sweet Home Alabam...|      Comedy|Romance|
|    368|   100|   4.5|     Maverick (1994)|Adventure|Comedy|...|
|    934|   100|   4.5|Father of the Bri...|              Comedy|
|    539|   100|   4.5|Sleepless in Seat...|Comedy|Drama|Romance|
|     16|   100|   4.5|       Casino (1995)|         Crime|Drama|
|    553|   100|   4.5|    Tombstone (1993)|Action|Drama|Western|
+-------+------+------+--------------------+--------------------+

